Nama : Ken Foster Morintoh\
NIM  : 672020175\
Kelas : Big Data Analysis\
Tugas : Logistic Regression (Tugas Akhir Semester)


**Deskripsi Tugas** : 

Buat Program Klasifikasi dengan Logistic Regression dengan SPARK\
Dataset= titanic.csv\
Split train:test -> 70:30\
Kolom features (X): Pclass, Sex, Age,  SibSp, Parch, Fare, Embarked, Gelar\
Kolom target (Y): Survived

Program (boleh) dibuat dengan kaggle\
Kirim file.ipynb ke suryasatriya@uksw.edu\
Batas Waktu: 24 Juni 2024, 24.00 WIB


In [63]:
import pyspark
from pyspark.sql import SparkSession

In [64]:
spark = SparkSession.builder.appName('Titanic').getOrCreate()

In [65]:
df= spark.read.csv("titanic.csv", inferSchema=True, header=True)

df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

**Menambahkan feature Gelar dan mengisikan valuenya dari ekstrasi dari kolom name**

- menggunakan regex untuk mendapatkan nilai gelar dari kolom Name yang akan dimasukan pada kolom(feature) Gelar.\
- Reggex akan menemukan mengektrak kolom title dari kolom namA\
    regexp_extract adalah fungsi yang disediakan oleh PySpark untuk mengekstraksi substring yang cocok dari kolom berdasarkan pola ekspresi reguler. Ini mengembalikan nilai yang cocok dengan grup penangkap tertentu.
   - Argumen pertama adalah kolom tempat ekspresi reguler akan diterapkan. Di sini, df["Name"], yang merujuk ke kolom "Name" di DataFrame.
   - Argumen kedua adalah pola ekspresi reguler sebagai string mentah (r"..."). Dalam hal ini, polanya adalah r"\b(\w+)\."
- untuk membuat kolom baru cukup menggunakan withColumn.


In [66]:
#menambah Kolom gelar dan memasukan valuenya
from pyspark.sql.functions import regexp_extract

df = df.withColumn("Gelar", regexp_extract(df["Name"], r"\b(\w+)\.", 1))

#Print dataset.
df.show(truncate=False)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|Gelar |
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22.0|1    |0    |A/5 21171       |7.25   |NULL |S       |Mr    |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |Mrs   |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26.0|0    |0    |STON/O2. 3101282|7.925  |NULL |S       |Miss  |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath 

Berdasarkan data diatas, penambahan kolom dengan gelar berhasil ditambahkan.

In [67]:

df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked',
 'Gelar']

In [68]:

df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Gelar: string (nullable = true)



**Feature selection berdasarkan instruksi : Kolom features (X): Pclass, Sex, Age,  SibSp, Parch, Fare, Embarked, Gelar**


In [69]:

rm_columns = df.select(['Survived','Pclass', 
                       'Sex','Age','SibSp', 
                       'Parch','Fare','Embarked', 'Gelar']) 
  
# menghapus data null
result = rm_columns.na.drop() 
  
  
result.show()

+--------+------+------+----+-----+-----+-------+--------+------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked| Gelar|
+--------+------+------+----+-----+-----+-------+--------+------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|    Mr|
|       1|     1|female|38.0|    1|    0|71.2833|       C|   Mrs|
|       1|     3|female|26.0|    0|    0|  7.925|       S|  Miss|
|       1|     1|female|35.0|    1|    0|   53.1|       S|   Mrs|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|    Mr|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|    Mr|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|Master|
|       1|     3|female|27.0|    0|    2|11.1333|       S|   Mrs|
|       1|     2|female|14.0|    1|    0|30.0708|       C|   Mrs|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|  Miss|
|       1|     1|female|58.0|    0|    0|  26.55|       S|  Miss|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|    Mr|
|       0|

Dapat terlihat dataframe diatas masih ada tiga data yang bertipe non numerik. Untuk itu perlu penkonversian sehingga data menjadi numerik semua agar dapat dilakukan logistic regression.

**1. Mengubah Kolom Sex menjadi numerik**

- saya menggunakan library pyspark.sql library untuk menemukan dan mengubah kata male menjadi 1 dan 0 untuk lainnya pada kolom Sex.
- menyimpannya dalam result_num_sex

In [70]:
from pyspark.sql.functions import col, when

result_num_sex = result.withColumn('Sex', when(col("Sex") == "male", 1).otherwise(0))
  

In [71]:
result_num_sex.show()

+--------+------+---+----+-----+-----+-------+--------+------+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|Embarked| Gelar|
+--------+------+---+----+-----+-----+-------+--------+------+
|       0|     3|  1|22.0|    1|    0|   7.25|       S|    Mr|
|       1|     1|  0|38.0|    1|    0|71.2833|       C|   Mrs|
|       1|     3|  0|26.0|    0|    0|  7.925|       S|  Miss|
|       1|     1|  0|35.0|    1|    0|   53.1|       S|   Mrs|
|       0|     3|  1|35.0|    0|    0|   8.05|       S|    Mr|
|       0|     1|  1|54.0|    0|    0|51.8625|       S|    Mr|
|       0|     3|  1| 2.0|    3|    1| 21.075|       S|Master|
|       1|     3|  0|27.0|    0|    2|11.1333|       S|   Mrs|
|       1|     2|  0|14.0|    1|    0|30.0708|       C|   Mrs|
|       1|     3|  0| 4.0|    1|    1|   16.7|       S|  Miss|
|       1|     1|  0|58.0|    0|    0|  26.55|       S|  Miss|
|       0|     3|  1|20.0|    0|    0|   8.05|       S|    Mr|
|       0|     3|  1|39.0|    1|    5| 31.275|       S|

**2. Mengubah kolom Embarked menjadi numerik**
- pertama-tama, untuk memastikan jumlah kategoti yang ada pada embarked, saya melakukan cek nilai unik pada kolom embarked
- setelah didapatkan, saya membuatkan mapping untuk setiap kategori pada embarked.
- mapping tersebut yang nantinya digunakan untuk mengubah nilai kategori embarked menjadi numerik
- Kemudian terakhir menerapakan metode when-otherwise unutk mengubah nilai

In [74]:
#memilih kolom
distinct_values = result_num_sex.select('Embarked').distinct()

# hitung kategori unik 
unique_count = distinct_values.count()
print(f"Jumlah kategori unik dalam feature 'Embarked': {unique_count}")

Jumlah kategori unik dalam feature 'Embarked': 3


In [76]:

# menentukan mapping setiap value pada embarked
category_mapping = {"Q": 1, "C": 2, "S": 3}

# melakukan transpormasi menggunakan when otherwise method
result_num_sex_embarked = result_num_sex.withColumn("Embarked", 
                           when(col("Embarked") == "Q", category_mapping["Q"])
                           .when(col("Embarked") == "C", category_mapping["C"])
                           .when(col("Embarked") == "S", category_mapping["S"])
                           .otherwise(0)) 


In [77]:
result_num_sex_embarked.show()

+--------+------+---+----+-----+-----+-------+--------+------+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|Embarked| Gelar|
+--------+------+---+----+-----+-----+-------+--------+------+
|       0|     3|  1|22.0|    1|    0|   7.25|       3|    Mr|
|       1|     1|  0|38.0|    1|    0|71.2833|       2|   Mrs|
|       1|     3|  0|26.0|    0|    0|  7.925|       3|  Miss|
|       1|     1|  0|35.0|    1|    0|   53.1|       3|   Mrs|
|       0|     3|  1|35.0|    0|    0|   8.05|       3|    Mr|
|       0|     1|  1|54.0|    0|    0|51.8625|       3|    Mr|
|       0|     3|  1| 2.0|    3|    1| 21.075|       3|Master|
|       1|     3|  0|27.0|    0|    2|11.1333|       3|   Mrs|
|       1|     2|  0|14.0|    1|    0|30.0708|       2|   Mrs|
|       1|     3|  0| 4.0|    1|    1|   16.7|       3|  Miss|
|       1|     1|  0|58.0|    0|    0|  26.55|       3|  Miss|
|       0|     3|  1|20.0|    0|    0|   8.05|       3|    Mr|
|       0|     3|  1|39.0|    1|    5| 31.275|       3|

**3. Mengubah kolom Gelar menjadi numerik**
- Melihat kategori yang ada 
- Mapping setiap kategori ke numerik
- Lalu menggunakan metode when-otherwise untuk melakukan konversi

In [78]:

from pyspark.sql.functions import count
category_counts = result_num_sex.groupBy("Gelar").agg(count("*").alias("count"))

category_counts.show()

+--------+-----+
|   Gelar|count|
+--------+-----+
|     Don|    1|
|    Miss|  145|
|Countess|    1|
|     Col|    2|
|     Rev|    6|
|    Lady|    1|
|  Master|   36|
|     Mme|    1|
|    Capt|    1|
|      Mr|  398|
|      Dr|    6|
|     Mrs|  107|
|     Sir|    1|
|Jonkheer|    1|
|    Mlle|    2|
|   Major|    2|
|      Ms|    1|
+--------+-----+



Mengubah Nilai Kategorikal pada kolom Gelar
- setiap kategori yang ditemukan akan digantikan dengan angka pada mapping yang kita buat

In [81]:
#mengubaha Miss Mr 
category_mapping1 = {
    'Capt': 1, 'Col': 2, 'Countess': 3, 'Don': 4, 
    'Dr': 5, 'Jonkheer': 6, 'Lady': 7, 'Major': 8, 
    'Master': 9, 'Miss': 10, 'Mlle': 11, 'Mme': 12, 
    'Mr': 13, 'Mrs': 14, 'Ms': 15, 'Rev': 16, 'Sir': 17
}


# Apply transformation using 'when' and 'otherwise'
my_final_dataset = result_num_sex_embarked.withColumn("Gelar", when(col("Gelar") == "Capt", category_mapping1["Capt"])
                           .when(col("Gelar") == "Col", category_mapping1["Col"])
                           .when(col("Gelar") == "Countess", category_mapping1["Countess"])
                           .when(col("Gelar") == "Don", category_mapping1["Don"])
                           .when(col("Gelar") == "Dr", category_mapping1["Dr"])
                           .when(col("Gelar") == "Jonkheer", category_mapping1["Jonkheer"])
                           .when(col("Gelar") == "Lady", category_mapping1["Lady"])
                           .when(col("Gelar") == "Major", category_mapping1["Major"])
                           .when(col("Gelar") == "Master", category_mapping1["Master"])
                           .when(col("Gelar") == "Miss", category_mapping1["Miss"])
                           .when(col("Gelar") == "Mlle", category_mapping1["Mlle"])
                           .when(col("Gelar") == "Mme", category_mapping1["Mme"])
                           .when(col("Gelar") == "Mr", category_mapping1["Mr"])
                           .when(col("Gelar") == "Mrs", category_mapping1["Mrs"])
                           .when(col("Gelar") == "Ms", category_mapping1["Ms"])
                           .when(col("Gelar") == "Rev", category_mapping1["Rev"])
                           .when(col("Gelar") == "Sir", category_mapping1["Sir"])
                         )
my_final_dataset.show()


+--------+------+---+----+-----+-----+-------+--------+-----+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|Embarked|Gelar|
+--------+------+---+----+-----+-----+-------+--------+-----+
|       0|     3|  1|22.0|    1|    0|   7.25|       3|   13|
|       1|     1|  0|38.0|    1|    0|71.2833|       2|   14|
|       1|     3|  0|26.0|    0|    0|  7.925|       3|   10|
|       1|     1|  0|35.0|    1|    0|   53.1|       3|   14|
|       0|     3|  1|35.0|    0|    0|   8.05|       3|   13|
|       0|     1|  1|54.0|    0|    0|51.8625|       3|   13|
|       0|     3|  1| 2.0|    3|    1| 21.075|       3|    9|
|       1|     3|  0|27.0|    0|    2|11.1333|       3|   14|
|       1|     2|  0|14.0|    1|    0|30.0708|       2|   14|
|       1|     3|  0| 4.0|    1|    1|   16.7|       3|   10|
|       1|     1|  0|58.0|    0|    0|  26.55|       3|   10|
|       0|     3|  1|20.0|    0|    0|   8.05|       3|   13|
|       0|     3|  1|39.0|    1|    5| 31.275|       3|   13|
|       

**Melakukan vector assembler**

In [83]:

from pyspark.ml.feature import VectorAssembler, OneHotEncoder 

# Encode kolom Sex 
sexEncode = OneHotEncoder(inputCol='Sex', 
                               outputCol='SexVec') 
# Encode kolom embarked
embarkEncode = OneHotEncoder(inputCol='Embarked', 
                               outputCol='EmbarkVec') 
# encode kolom Gelar 
gelarEncode = OneHotEncoder(inputCol='Gelar',
                            outputCol='GelarVec')
    
# Melakukan Vektorisasi data menjadi kolom baru "feature"
assembler = VectorAssembler(inputCols=['Pclass', 
                                       'SexVec','Age', 
                                       'SibSp','Parch', 
                                       'Fare','EmbarkVec', 'GelarVec'], 
                                    outputCol='features')


**Pipelining**

In [84]:
#Menginport pipeline dan model yang saya gunakan 
from pyspark.ml import Pipeline 
from pyspark.ml.classification import LogisticRegression 

**Target Kolom Y ("Survived")**

In [89]:
log_reg = LogisticRegression(featuresCol='features', 
                             labelCol='Survived') 
  
# Membuat papeline
pipelineku = Pipeline(stages=[sexEncode, embarkEncode, gelarEncode,
                            assembler, log_reg]) 

**Split train:test 70:30** 

In [90]:
# split data 70:30
train_data, test_data = my_final_dataset.randomSplit([0.7, .3]) 
  
# fiting model
fit_model = pipelineku.fit(train_data) 
  
# menyimpan hasil fitting di "result"
results = fit_model.transform(test_data) 
  
results.show()

+--------+------+---+----+-----+-----+-------+--------+-----+---------+-------------+---------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|Embarked|Gelar|   SexVec|    EmbarkVec|       GelarVec|            features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+-------+--------+-----+---------+-------------+---------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  1|18.0|    1|    0|  108.9|       2|   13|(1,[],[])|(3,[2],[1.0])|(17,[13],[1.0])|(26,[0,2,3,5,8,22...|[-0.3087954431551...|[0.42340878472684...|       1.0|
|       0|     1|  1|21.0|    0|    1|77.2875|       3|   13|(1,[],[])|    (3,[],[])|(17,[13],[1.0])|(26,[0,2,4,5,22],...|[-0.2115477733081...|[0.44730941304637...|       1.0|
|       0|     1|  1|27.0|    0|    2|  211.5|       2|   13|(1,[],[])|(3,[2],[1.0])|(17,[13],[1.0])|(26,[0,2,4,5,8,22..

**Evaluasi Akurasi Model**

In [91]:

from pyspark.ml.evaluation import BinaryClassificationEvaluator 
my_res = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived') 
# Evaluasi prediksi
ROC_AUC = my_res.evaluate(results)
ROC_AUC

0.805757874015748

**Modelnya cukup baik dengan akurasi 78-80%**

**Tunning hyper Parameter menggunakan String indexter untuk konversi ke numerik**
- disini pengubahan nilai numerik dilakukan oleh StringIndexter tanpa bantuan kita untuk mengubah data string ke numerik.


In [92]:

from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder 
  
# convert kolom sex
sexIdx = StringIndexer(inputCol='Sex', 
                               outputCol='SexIndex') 
sexEncode = OneHotEncoder(inputCol='SexIndex', 
                               outputCol='SexVec') 
  
# conver kolom embark
embarkIdx = StringIndexer(inputCol='Embarked', 
                               outputCol='EmbarkIndex') 
embarkEncode = OneHotEncoder(inputCol='EmbarkIndex', 
                               outputCol='EmbarkVec') 

#konver kolom gelar
gelarIdx = StringIndexer(inputCol='Gelar', 
                               outputCol='GelarIndex') 
gelarEncode = OneHotEncoder(inputCol='GelarIndex', 
                               outputCol='GelarVec') 


# Melakukan vektorisasi data kedalam kolom baru "features"
assembler2 = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch', 'Fare','EmbarkVec', 'GelarVec'], outputCol='features') 

In [93]:
from pyspark.ml import Pipeline 
from pyspark.ml.classification import LogisticRegression 

In [94]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Survived') 
  
# membuat pipelin
pipeline= Pipeline(stages=[sexIdx, embarkIdx, gelarIdx, sexEncode, embarkEncode, gelarEncode, assembler2, log_reg]) 

In [97]:
#membagi data 70:30 persen 
train_data, test_data = my_final_dataset.randomSplit([0.7, .3]) 
  
# fiting mmodel kdealam data train
fit_model = pipeline.fit(train_data) 
  
# menyimpan hasil fit ke "Hasil"
Hasil = fit_model.transform(test_data) 



In [98]:
Hasil.show()

+--------+------+---+----+-----+-----+-------+--------+-----+--------+-----------+----------+-------------+-------------+--------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|Sex| Age|SibSp|Parch|   Fare|Embarked|Gelar|SexIndex|EmbarkIndex|GelarIndex|       SexVec|    EmbarkVec|      GelarVec|            features|       rawPrediction|         probability|prediction|
+--------+------+---+----+-----+-----+-------+--------+-----+--------+-----------+----------+-------------+-------------+--------------+--------------------+--------------------+--------------------+----------+
|       0|     1|  0| 2.0|    1|    2| 151.55|       3|   10|     1.0|        0.0|       1.0|    (1,[],[])|(2,[0],[1.0])|(16,[1],[1.0])|(24,[0,2,3,4,5,6,...|[-3.0224584861764...|[0.04642152433586...|       1.0|
|       0|     1|  1|27.0|    0|    2|  211.5|       2|   13|     0.0|        1.0|       0.0|(1,[0],[1.0])|(2,[1],[1.0])|(16,[0],[1.0])|(24,[0,1,2,4,5,7,...

Evaluasi model dimana saya menggunakan string indexter

In [101]:

from pyspark.ml.evaluation import BinaryClassificationEvaluator 

In [102]:
 
res = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='Survived') 

# Evaluating the AUC on results 
ROC_AUC2 = res.evaluate(Hasil) 

ROC_AUC2

0.8425693864290354

Disini saya mendapatkan akurasi yang lebih baik. Dapat dikatakan tunning berhasil